In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import importlib.util

In [ ]:
#print(sys.path)

In [3]:
sys.path.insert(1, "C:/Users/Mi/Documents/Diploma/GMDH/build/Release")
sys.path.insert(1, "/home/mikhail-xnor/Projects/GMDH/build")

In [4]:
import gmdhpy as gm

In [5]:
dir(gm)

['Combi',
 'Criterion',
 'CriterionType',
 'GmdhModel',
 'Mia',
 'Multi',
 'ParallelCriterion',
 'PolynomialType',
 'Ria',
 'SequentialCriterion',
 'Solver',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'split_data',
 'splitted_data',
 'time_series_transformation']

In [5]:
def data_preparation(x, lags, validate_size, test_size=0):
    #print("x: ", x)
    x_data = list()
    y_data = list()
    for i in range(len(x) - lags):
        x_data.append(list(x.values[i:i+lags]))
        y_data.append(x.values[i+lags])
    x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=validate_size, shuffle=False)
    x_train = np.array(x_train)
    x_val = np.array(x_val)
    #print("x_train:\n", x_train)
    #print("y_train:\n", y_train)
    #print("x_val:\n", x_val)
    #print("y_val:\n", y_val)
    if test_size > 0:
        x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=test_size, shuffle=False)
        x_train = np.array(x_train)
        x_test = np.array(x_test)
        return x_train, y_train, x_test, y_test, x_val, y_val
    else:
        return x_train, y_train, x_val, y_val

def print_polynom(polynom, coeffs):
    best_polynom_str = "y ="
    for i in range(len(polynom)):
        if coeffs[i] > 0:
            if i > 0:
                best_polynom_str += " + "
            else:
                best_polynom_str += " "
        else:
            best_polynom_str += " - "
        best_polynom_str += str(abs(coeffs[i]))
        if i != len(coeffs) - 1:
            best_polynom_str += "*x" + str(polynom[i] + 1)
    print(best_polynom_str)

In [6]:
def regression(x, lags, validate_size):
    x_train, y_train, x_val, y_val = data_preparation(x, lags, validate_size)
    lr = LinearRegression(n_jobs=-1)
    lr.fit(x_train, y_train)
    coeffs = list(lr.coef_)
    coeffs.append(lr.intercept_)
    coeffs = [round(coeff, 6) for coeff in coeffs]
    y_pred = lr.predict(x_val)
    error = round(mean_squared_error(y_pred, y_val), 6)
    print_polynom(range(lags + 1), coeffs)
    return y_pred

In [7]:
def regression_combi(x, lags, validate_size, test_size):
    x_train, y_train, x_test, y_test, x_val, y_val = data_preparation(x, lags, validate_size, test_size)
    lr = LinearRegression(n_jobs=-1)
    last_error = 100000000000000
    best_coeffs = []
    best_polynom = []
    for level in range(1, x_train.shape[1] + 1):
        errors = []
        for comb in combinations(range(x_train.shape[1]), level):
            lr.fit(x_train[:, comb], y_train)
            coeffs = list(lr.coef_)
            coeffs.append(lr.intercept_)
            coeffs = [round(coeff, 6) for coeff in coeffs]
            y_pred = lr.predict(x_test[:, comb])
            error = round(mean_squared_error(y_pred, y_test), 6)
            comb = list(comb)
            comb.append(lags)
            errors.append((error, coeffs, comb))
        errors.sort()
        #print(errors[0])
        if last_error > errors[0][0]:
            last_error = errors[0][0]
            best_coeffs = errors[0][1]
            best_polynom = errors[0][2]
        else:
            break
    best_polynom_str = "y ="
    print_polynom(best_polynom, best_coeffs)
    y_pred = lr.fit(x_train[:, best_polynom[:-1]], y_train).predict(x_val[:, best_polynom[:-1]])
    return y_pred

In [ ]:
"""%%time
data = pd.read_csv("Sberbank.csv")['close'][:-50000]
lags = 10
test_size = 0.33
validate_size = 0.2

mat, vec = gm.time_series_transformation(data, lags)
splited_data = gm.split_data(mat, vec, validate_size, False, 0)"""

In [6]:
def print_metrics(y_test, y_pred):
    print(f"R^2: {r2_score(y_test, y_pred)}")
    print(f"MSE: {mean_squared_error(y_test, y_pred)}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred)}")

In [7]:
data = pd.read_csv("laptop_price.csv")
#data = data[data.columns[data.dtypes != object]].drop(['laptop_ID', 'Memory2_GB'], axis=1)
data = data.drop(['laptop_ID', 'Memory2_GB'], axis=1)
data = data.drop(data[data['Ram_GB'] == 64].index)
test_size = 0.2
validate_size = 0.2
data.head()

,Company,Product,TypeName,Inches,Ram_GB,OpSys,Weight_kg,Price_euros,Touchscreen,ScreenWidth,...,Cpu_producer,Cpu_series,Cpu_model,Cpu_GHz,Gpu_producer,Gpu_model,Gpu_series,Memory1_GB,Memory1_type,Memory2_type
0,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1339.69,0,2560,...,Intel,-,Core i5,2.3,Intel,Iris Plus Graphics,640,128,SSD,-
1,Apple,Macbook Air,Ultrabook,13.3,8,macOS,1.34,898.94,0,1440,...,Intel,-,Core i5,1.8,Intel,HD Graphics,6000,128,Flash Storage,-
2,HP,250 G6,Notebook,15.6,8,No OS,1.86,575.00,0,1920,...,Intel,7200U,Core i5,2.5,Intel,HD Graphics,620,256,SSD,-
3,Apple,MacBook Pro,Ultrabook,15.4,16,macOS,1.83,2537.45,0,2880,...,Intel,-,Core i7,2.7,AMD,Radeon Pro,455,512,SSD,-
4,Apple,MacBook Pro,Ultrabook,13.3,8,macOS,1.37,1803.60,0,2560,...,Intel,-,Core i5,3.1,Intel,Iris Plus Graphics,650,256,SSD,-


In [8]:
category_cols = data.columns[data.dtypes == object]
for col in category_cols:
    print(f'{col}: {data[col].unique().size}')

Company: 19
Product: 617
TypeName: 6
OpSys: 9
Cpu_producer: 3
Cpu_series: 88
Cpu_model: 20
Gpu_producer: 4
Gpu_model: 26
Gpu_series: 75
Memory1_type: 4
Memory2_type: 4


In [10]:
data = pd.get_dummies(data=data.drop(['Product'], axis=1), columns=category_cols.drop(['Product']))
splited_data = gm.split_data(data.drop(['Price_euros'], axis=1), data['Price_euros'])
scaler = MinMaxScaler().fit(splited_data.x_train)
splited_data.x_train = scaler.transform(splited_data.x_train);
splited_data.x_test = scaler.transform(splited_data.x_test);

In [ ]:
%%time
combi = gm.Combi()
combi.fit(splited_data.x_train, splited_data.y_train, gm.Criterion(gm.CriterionType.regularity, gm.Solver.accurate), test_size, True, 17, 0, -1, 1, 0)
print(combi.get_best_polynomial())
y_pred_combi = combi.predict(splited_data.x_test)

LEVEL 1  [==============================] 100% :00s] (103 combinations)                                             


<timed exec>:2: Warning: 
Warning! The input number of pAverage is incorrect!
The default value is used (pAverage = 1)!



LEVEL 2  [==============================] 100% :00s] (5253 combinations)                                            
LEVEL 3  [==============================] 100% :05s] (176851 combinations)                                          


In [204]:
print_metrics(splited_data.y_test, y_pred_combi)

R^2: 0.5656537104846866
MSE: 194191.66116391218
MAE: 306.37671592064873


In [11]:
%%time
multi = gm.Multi()
multi.fit(splited_data.x_train, splited_data.y_train, gm.Criterion(gm.CriterionType.sym_regularity, gm.Solver.accurate), 3, test_size, True, 17, 0, -1, 1, 0);
print(multi.get_best_polynomial())
y_pred_multi = multi.predict(splited_data.x_test)

LEVEL 1  [=========================] 100% :00s] (266 combinations) error=209191719.530470                           


<timed exec>:2: Warning: 
Warning! The input number of pAverage is incorrect!
The default value is used (pAverage = 1)!



LEVEL 2  [=========================] 100% :00s] (792 combinations) error=173810086.681330                           
LEVEL 3  [=========================] 100% :00s] (789 combinations) error=154415641.672885                           
LEVEL 4  [=========================] 100% :00s] (787 combinations) error=138024979.680430                           
LEVEL 5  [=========================] 100% :00s] (784 combinations) error=127854575.806107                           
LEVEL 6  [=========================] 100% :00s] (781 combinations) error=120050219.329098                           
LEVEL 7  [=========================] 100% :00s] (778 combinations) error=116557494.179646                           
LEVEL 8  [=========================] 100% :00s] (774 combinations) error=113049038.032135                           
LEVEL 9  [=========================] 100% :00s] (772 combinations) error=108857010.953786                           
LEVEL 10 [=========================] 100% :00s] (768 combination

LEVEL 69 [=========================] 100% :01s] (592 combinations) error=78168620.456250                            
LEVEL 70 [=========================] 100% :01s] (588 combinations) error=78156711.826878                            
LEVEL 71 [=========================] 100% :01s] (585 combinations) error=78146568.186673                            
LEVEL 72 [=========================] 100% :01s] (582 combinations) error=78137716.049204                            
LEVEL 73 [=========================] 100% :01s] (579 combinations) error=78125808.254833                            
LEVEL 74 [=========================] 100% :02s] (576 combinations) error=78117522.486968                            
LEVEL 75 [=========================] 100% :02s] (573 combinations) error=78109698.177175                            
LEVEL 76 [=========================] 100% :02s] (570 combinations) error=78103662.178796                            
LEVEL 77 [=========================] 100% :02s] (567 combination

In [19]:
print_metrics(splited_data.y_test, y_pred_multi)

R^2: 0.6868207650056062
MSE: 140019.14452514367
MAE: 259.38463752355136


In [11]:
try:
    y_pred_multi = multi.predict(np.delete(splited_data.x_test, -1, 1))
except Exception as err:
    print(err)

Input data number of cols is not match number of cols of fitted data!


In [29]:
%%time
mia = gm.Mia()
mia.fit(splited_data.x_train, splited_data.y_train, gm.Criterion(gm.CriterionType.sym_regularity, gm.Solver.accurate), 20, gm.PolynomialType.linear_cov, test_size, 1, 0, 3, -10, 1);
#print(mia.get_best_polynomial())
y_pred_mia = mia.predict(splited_data.x_test)

<timed exec>:2: Warning: 
Warning! The input number of threads is incorrect!
The default value is used (threads = 1)!



LEVEL 1  [=========================] 100% :05s] (35245 combinations) error=179528354.233117                         
LEVEL 2  [=========================] 100% :00s] (190 combinations) error=165130680.782079                           
LEVEL 3  [=========================] 100% :00s] (190 combinations) error=160080513.143528                           
LEVEL 4  [=========================] 100% :00s] (190 combinations) error=159273631.702171                           
LEVEL 5  [=========================] 100% :00s] (190 combinations) error=158950764.521376                           
LEVEL 6  [=========================] 100% :00s] (190 combinations) error=158755067.219075                           
LEVEL 7  [=========================] 100% :00s] (190 combinations) error=158698168.380706                           
LEVEL 8  [=========================] 100% :00s] (190 combinations) error=158633072.539281                           
LEVEL 9  [=========================] 100% :00s] (190 combination

In [40]:
print_metrics(splited_data.y_test, y_pred_mia)

R^2: 0.5708349269634132
MSE: 191875.19372965256
MAE: 309.870941863932


In [80]:
data.describe()

,Inches,Ram_GB,Weight_kg,Price_euros,Touchscreen,ScreenWidth,ScreenHeight,Cpu_GHz,Memory1_GB,Company_Acer,...,Gpu_series_W5130M,Gpu_series_W6150M,Memory1_type_Flash Storage,Memory1_type_HDD,Memory1_type_Hybrid,Memory1_type_SSD,Memory2_type_-,Memory2_type_HDD,Memory2_type_Hybrid,Memory2_type_SSD
count,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,...,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000,1244.000000
mean,15.029904,8.403537,2.044035,1130.563055,0.144695,1897.607717,1072.469453,2.304196,446.774920,0.080386,...,0.000804,0.000804,0.055466,0.284566,0.006431,0.653537,0.836817,0.158360,0.001608,0.003215
std,1.414841,4.884624,0.668770,700.047321,0.351934,492.807819,283.722976,0.502959,367.634894,0.271999,...,0.028352,0.028352,0.228980,0.451389,0.079967,0.476034,0.369681,0.365225,0.040080,0.056636
min,10.100000,2.000000,0.690000,174.000000,0.000000,1366.000000,768.000000,0.900000,8.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,14.000000,4.000000,1.500000,599.000000,0.000000,1600.000000,900.000000,2.000000,256.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,15.600000,8.000000,2.040000,986.500000,0.000000,1920.000000,1080.000000,2.500000,256.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,15.600000,8.000000,2.310000,1491.450000,0.000000,1920.000000,1080.000000,2.700000,512.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,18.400000,32.000000,4.700000,6099.000000,1.000000,3840.000000,2160.000000,3.600000,2048.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [1]:
%%time
ria = gm.Ria()
ria.fit(splited_data.x_train, splited_data.y_train, gm.Criterion(gm.CriterionType.regularity, gm.Solver.accurate), 5, gm.PolynomialType.linear, test_size, 0, 0, 1, -2, 1, 0);
#print(ria.get_best_polynomial())
y_pred_ria = ria.predict(splited_data.x_test)

NameError: name 'gm' is not defined

In [51]:
print_metrics(splited_data.y_test, y_pred_ria)

R^2: 0.6126159034652368
MSE: 173195.3582440234
MAE: 284.66992742385145


In [ ]:
%%time
y_pred_lr_combi = regression_combi(data, lags, validate_size, test_size)

In [ ]:
%time
y_pred_lr = regression(data, lags, validate_size)

In [ ]:
#np.savetxt("Sber.csv", np.asarray(data.values), delimiter=",")

In [ ]:
n = 20
plt.figure(figsize=(17, 6))
sns.lineplot(x=np.arange(n), y=splited_data.y_test[:n], label='original');
sns.lineplot(x=np.arange(n), y=y_pred_combi[:n], label='combi');
sns.lineplot(x=np.arange(n), y=y_pred_multi[:n], label='multi');
sns.lineplot(x=np.arange(n), y=y_pred_mia[:n], label='mia');
sns.lineplot(x=np.arange(n), y=y_pred_ria[:n], label='ria');
#sns.lineplot(x=np.arange(n), y=y_pred_lr_combi[:n], label='regr_combi');
#sns.lineplot(x=np.arange(n), y=y_pred_lr[:n], label='regr');

In [ ]:
def stat(y_real, y_pred):
    y_real_diff = np.diff(y_real)
    y_pred_diff = np.diff(y_pred)
    print("right:", ((y_real_diff * y_pred_diff > 0)).sum())
    print("wrong:", ((y_real_diff * y_pred_diff < 0)).sum())
    print("zero:", ((y_real_diff * y_pred_diff == 0)).sum())

In [ ]:
print("COMBI")
stat(splited_data.y_test, y_pred_combi)
print("\nMULTI")
stat(splited_data.y_test, y_pred_multi)
print("\nMIA")
stat(splited_data.y_test, y_pred_mia)
print("\nRIA")
stat(splited_data.y_test, y_pred_ria)